In [1]:
import tkinter as tk
from tkinter import colorchooser, filedialog, simpledialog
from PIL import Image, ImageDraw, ImageTk

In [2]:
class PaintApp:
    def __init__(self, root):
        # Initialize the main window and title
        self.root = root
        self.root.title("Roya's Painting Application")
        
        # Set default brush attributes
        self.brush_color = "yellow"
        self.brush_size = 5
        self.brush_shape = "oval"
        self.undo_stack = []
        self.redo_stack = []
        self.is_eraser = False
        self.text_position = None

        # Create and pack the main frame
        self.main_frame = tk.Frame(root, padx=5, pady=5)
        self.main_frame.pack(side=tk.TOP,fill='both')

        # Create and place the canvas widget for drawing
        self.canvas = tk.Canvas(self.main_frame, bg="black", width=900, height=400, bd=7, relief=tk.FLAT)  #SUNKEN,GROOVE,RIDGE,FLAT
        self.canvas.grid(row=0, column=0, columnspan=11, pady=5)

        # Create and place the control frame with buttons and sliders
        self.control_frame = tk.Frame(self.main_frame, padx=25, pady=25)
        self.control_frame.grid(row=1, column=0, columnspan=9, sticky="nw")

        # Create button to choose brush color
        self.color_button = tk.Button(self.control_frame, text="Choose Color:", command=self.choose_color, bg='black', font=("Times New Roman", 12), fg="white")
        self.color_button.grid(row=0, column=0, padx=5, sticky='ew')

        # Create label and scale for brush size
        self.size_label = tk.Label(self.control_frame, text="Brush Size:", bg='white', font=("Times New Roman", 10,"bold"), fg="black")
        self.size_label.grid(row=0, column=9, padx=5, sticky='ew')

        self.size_scale = tk.Scale(self.control_frame, from_=1, to=10, orient=tk.VERTICAL, length=100)
        self.size_scale.set(self.brush_size)
        self.size_scale.grid(row=0, column=10, padx=5, sticky='ew')
 

        # Create menu button for selecting brush shape
        self.shape_menu = tk.Menubutton(self.control_frame,text="Brush Shape" , relief=tk.RAISED, bg='black', font=("Times New Roman", 12), fg="white")
        self.shape_menu.menu = tk.Menu(self.shape_menu, tearoff=0,bg="black",fg="white")
        self.shape_menu["menu"] = self.shape_menu.menu
        self.shape_menu.menu.add_radiobutton(label="Ellipse", command=lambda : self.set_brush_shape("oval"))
        self.shape_menu.menu.add_radiobutton(label="Rectangle", command=lambda: self.set_brush_shape("rectangle"))
        self.shape_menu.grid(row=0, column=3, padx=5,pady=10, sticky='ew')

        # Create buttons for erasing, clearing, saving, undoing, and redoing actions
        self.erase_button = tk.Button(self.control_frame, text="Erase", command=self.toggle_eraser,bg='black', font=("Times New Roman", 12), fg="white")
        self.erase_button.grid(row=0, column=2, padx=5, sticky='ew')

        self.clear_button = tk.Button(self.control_frame, text="Clear", command=self.clear_canvas, bg='black', font=("Times New Roman", 12), fg="white")
        self.clear_button.grid(row=0, column=4, padx=5, sticky='ew')

        self.save_button = tk.Button(self.control_frame, text="Save", command=self.save_canvas, bg='black', font=("Times New Roman", 12), fg="white")
        self.save_button.grid(row=0, column=1, padx=5, sticky='ew')

        self.undo_button = tk.Button(self.control_frame, text="Undo", command=self.undo, bg='black', font=("Times New Roman", 12), fg="white")
        self.undo_button.grid(row=0, column=5, padx=5, sticky='ew')

        self.redo_button = tk.Button(self.control_frame, text="Redo", command=self.redo, bg='black', font=("Times New Roman", 12), fg="white")
        self.redo_button.grid(row=0, column=6, padx=5, sticky='ew')

        # Create buttons for adding images and text to the canvas
        self.add_image_button = tk.Button(self.control_frame, text="Add Image", command=self.add_image, bg='black', font=("Times New Roman", 12), fg="white")
        self.add_image_button.grid(row=0, column=7, padx=5, pady=5, sticky='ew')

        self.add_text_button = tk.Button(self.control_frame, text="Add Text", command=self.add_text, bg='black', font=("Times New Roman", 12), fg="white")
        self.add_text_button.grid(row=0, column=8, padx=5, pady=5, sticky='ew')

        # Bind mouse events for painting and setting text position
        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<Button-1>", self.set_text_position)

        # Initialize the image and drawing objects for saving
        self.image = Image.new("RGB", (900, 400), "black")
        self.draw = ImageDraw.Draw(self.image)

        # Set up the close button for the main window
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def toggle_eraser(self):
        # Toggle between eraser and brush modes
        self.is_eraser = not self.is_eraser
    
    def choose_color(self):
        # Open color chooser and set brush color
        self.is_eraser = False
        color = colorchooser.askcolor()[1]
        if color:
            self.brush_color = color
        else:
            print("No color selected")

    def set_brush_shape(self, shape):
        # Set the current brush shape
        self.brush_shape = shape

    def clear_canvas(self):
        # Clear the canvas and reset image
        self.canvas.delete("all")
        self.draw.rectangle([0, 0, 900, 400], fill="black")
        self.undo_stack.clear()
        self.redo_stack.clear()

    def paint(self, event):
        # Draw on the canvas based on brush attributes
        if self.is_eraser:
            self.erase(event)
        else:
            self.brush_size = self.size_scale.get()
            x1, y1 = (event.x - self.brush_size), (event.y - self.brush_size)
            x2, y2 = (event.x + self.brush_size), (event.y + self.brush_size)

            if self.brush_shape == "oval":
                item = self.canvas.create_oval(x1, y1, x2, y2, fill=self.brush_color, outline=self.brush_color)
            elif self.brush_shape == "rectangle":
                item = self.canvas.create_rectangle(x1, y1, x2, y2, fill=self.brush_color, outline=self.brush_color)

            self.undo_stack.append((item, self.canvas.coords(item), self.canvas.itemcget(item, "fill"), self.canvas.itemcget(item, "outline")))
            self.redo_stack.clear()

            self.update_image()

    def erase(self, event):
        # Remove items from the canvas where the eraser is applied
        self.brush_size = self.size_scale.get()
        x1, y1 = (event.x - self.brush_size), (event.y - self.brush_size)
        x2, y2 = (event.x + self.brush_size), (event.y + self.brush_size)

        overlapping_items = self.canvas.find_overlapping(x1, y1, x2, y2)
        for item in overlapping_items:
            self.canvas.delete(item)
            self.undo_stack = [entry for entry in self.undo_stack if entry[0] != item]

        self.update_image()

    def save_canvas(self):
        # Open file dialog to save the canvas image
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
        if file_path:
            self.image.save(file_path)

    def undo(self):
        # Undo the last action
        if self.undo_stack:
            item, coords, fill, outline = self.undo_stack.pop()
            self.redo_stack.append((item, coords, fill, outline))
            self.canvas.delete(item)

            self.update_image()

    def redo(self):
        # Redo the last undone action
        if self.redo_stack:
            item, coords, fill, outline = self.redo_stack.pop()
            if self.brush_shape == "oval":
                item = self.canvas.create_oval(*coords, fill=fill, outline=outline)
            elif self.brush_shape == "rectangle":
                item = self.canvas.create_rectangle(*coords, fill=fill, outline=outline)
            self.undo_stack.append((item, coords, fill, outline))

            self.update_image()

    def update_image(self):
    # Update the saved image to match the current canvas state
     #self.image.paste("black", [0, 0, 900, 400])
     for item in self.canvas.find_all():
        item_type = self.canvas.type(item)
        coords = self.canvas.coords(item)

        if item_type == "oval" or item_type == "rectangle":
            # Handle shapes
            color = self.canvas.itemcget(item, "fill")
            outline = self.canvas.itemcget(item, "outline")
            if len(coords) >= 4:  # Ensure coords has enough values
                if item_type == "oval":
                    self.draw.ellipse(coords, fill=color, outline=outline)
                elif item_type == "rectangle":
                    self.draw.rectangle(coords, fill=color, outline=outline)

        elif item_type == "image":
            # Handle image items
            file_path = self.canvas.image_refs.get(item)
            if file_path:
                img = Image.open(file_path)
                if len(coords) >= 4:  # Ensure coords has enough values
                    img = img.resize((int(coords[2] - coords[0]), int(coords[3] - coords[1])))
                    self.image.paste(img, (int(coords[0]), int(coords[1])))

        elif item_type == "text":
            # Handle text items
            color = self.canvas.itemcget(item, "fill")
            text = self.canvas.itemcget(item, "text")
            if len(coords) >= 2:  # Ensure coords has enough values
                self.draw.text((coords[0], coords[1]), text, fill=color)




    def add_image(self):
    # Add an image to the canvas
       file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif")])
       if file_path:
         width = simpledialog.askinteger("Image Width", "Enter the width of the image display:", minvalue=1)
         height = simpledialog.askinteger("Image Height", "Enter the height of the image display:", minvalue=1)
         if width and height:
            original_img = Image.open(file_path)  # Load the image without resizing
            self.original_image = original_img
            self.image_on_canvas = ImageTk.PhotoImage(original_img.resize((width, height)))  # Resize for display only
            img_id = self.canvas.create_image(0, 0, image=self.image_on_canvas, anchor=tk.NW)  # Place image on canvas
            self.canvas.image_refs = getattr(self.canvas, 'image_refs', {})
            self.canvas.image_refs[img_id] = file_path  # Store the image file path
            self.update_image()  # Ensure that the image is included in the saved image




    def set_text_position(self, event):
        # Set position for adding text
        self.text_position = (event.x, event.y)

    def add_text(self):
        # Add text to the canvas at the specified position
        if self.text_position:
            text = simpledialog.askstring("Input", "Enter the text to add:")
            if text:
                x, y = self.text_position
                self.canvas.create_text(x, y, text=text, fill=self.brush_color, font=("Times New Roman", 16))
                self.draw.text((x, y), text, fill=self.brush_color)
                self.update_image()  # Ensure text is included in the image

    def on_close(self):
        # Handle cleanup and closing the application
        self.root.destroy()

# Create the main window and run the application
root = tk.Tk()
app = PaintApp(root)

try:
    root.mainloop()
except KeyboardInterrupt:
    print("Painting application closed.")
except Exception:
    print("An unexpected error occurred.")
finally:
    print("Painting application is working.")

Painting application is working.
